In [1]:
import pandas as pd
import numpy as np
import regex as re
import datetime
from datetime import date

In [2]:
beer_data = pd.read_csv('beer_data.csv')
beer_data.head(3)

,brewery,beer_name,overall_score,style_score,rating,num_ratings,num_reviews,location,abv_percent,beer_style,description,served_in_list,cal_per_355_ml,date_added,url
0,Siren Craft Brew,Siren Caribbean Chocolate Mole Cake,98.0,70.0,3.99,22,20,"🇬🇧 Finchampstead, Berkshire, England",8.8,Stout - Imperial Flavored / Pastry,This brand new version of CCC takes inspiratio...,[],264.0,"November 20, 2021",https://www.ratebeer.com/beer/siren-caribbean-...
1,Brouwerij De Molen (Swinkels Family Brewers),De Molen Bommen & Granaten Burgundy BA,NaN,NaN,4.13,7,7,"🇳🇱 BODEGRAVEN, South Holland, Netherlands",11.1,Barley Wine / Wheat Wine / Rye Wine,"With a blow of Chinook, Challenger",['Snifter'],333.0,"December 10, 2021",https://www.ratebeer.com/beer/de-molen-bommen-...
2,Zagovor Brewery,Zagovor Sequoia Trail,NaN,NaN,3.50,1,1,"🇷🇺 Mytishchi, Moscow Oblast, Russia",6.0,IPA,IPA focused on HS-Sequoia hop blend paired wit...,"['Shaker', 'Tulip']",180.0,"February 6, 2022",https://www.ratebeer.com/beer/zagovor-sequoia-...


In [3]:
beer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485 entries, 0 to 1484
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brewery         1485 non-null   object 
 1   beer_name       1485 non-null   object 
 2   overall_score   584 non-null    float64
 3   style_score     584 non-null    float64
 4   rating          1485 non-null   float64
 5   num_ratings     1485 non-null   int64  
 6   num_reviews     1485 non-null   int64  
 7   location        1485 non-null   object 
 8   abv_percent     1476 non-null   float64
 9   beer_style      1485 non-null   object 
 10  description     1208 non-null   object 
 11  served_in_list  1485 non-null   object 
 12  cal_per_355_ml  1476 non-null   float64
 13  date_added      1268 non-null   object 
 14  url             1485 non-null   object 
dtypes: float64(5), int64(2), object(8)
memory usage: 174.1+ KB


## Step 1
Clean location column, specify by country only

In [4]:
# Choosing country only as location feature

beer_data['location'] = beer_data.location.map(lambda x: x.strip().split(',')[-1])
beer_data.head(3)

,brewery,beer_name,overall_score,style_score,rating,num_ratings,num_reviews,location,abv_percent,beer_style,description,served_in_list,cal_per_355_ml,date_added,url
0,Siren Craft Brew,Siren Caribbean Chocolate Mole Cake,98.0,70.0,3.99,22,20,England,8.8,Stout - Imperial Flavored / Pastry,This brand new version of CCC takes inspiratio...,[],264.0,"November 20, 2021",https://www.ratebeer.com/beer/siren-caribbean-...
1,Brouwerij De Molen (Swinkels Family Brewers),De Molen Bommen & Granaten Burgundy BA,NaN,NaN,4.13,7,7,Netherlands,11.1,Barley Wine / Wheat Wine / Rye Wine,"With a blow of Chinook, Challenger",['Snifter'],333.0,"December 10, 2021",https://www.ratebeer.com/beer/de-molen-bommen-...
2,Zagovor Brewery,Zagovor Sequoia Trail,NaN,NaN,3.50,1,1,Russia,6.0,IPA,IPA focused on HS-Sequoia hop blend paired wit...,"['Shaker', 'Tulip']",180.0,"February 6, 2022",https://www.ratebeer.com/beer/zagovor-sequoia-...


## Step 2
Revert served_in_list back to a list, then find unique values to use as columns for dummy variables

In [5]:
# served_in_list used to find unique values only
# Column was converted to string type when converted to .csv
# Take a string that looks like a Python object and turn it into that object
# Code credit to Alex Baransky

beer_data['served_in_list'] = beer_data.served_in_list.apply(eval)

In [6]:
# Take the series and return it into a nested list. Each inner list is the value for a single row
# Take a list of lists and flatten them so its one long list with no nested lists
# Code credit to Alex Baransky

served_in_unique = set(sum(beer_data.served_in_list.tolist(), []))

In [7]:
# Then, create new rows with these unique glass types
# Dummy variables indicate whether each beer is served in each glass type
# Code credit to Alex Baransky

for current_served_in in served_in_unique:
    beer_data[f"served_in_{re.sub('[ /]', '_', current_served_in).lower()}"] = beer_data.served_in_list.apply(lambda values: int(current_served_in in values))

beer_data.head(3)

,brewery,beer_name,overall_score,style_score,rating,num_ratings,num_reviews,location,abv_percent,beer_style,...,served_in_tumbler,served_in_stein,served_in_dimpled_mug,served_in_guinomi,served_in_weizen,served_in_yard,served_in_shaker,served_in_snifter,served_in_trappist_glass,served_in_stem_glass
0,Siren Craft Brew,Siren Caribbean Chocolate Mole Cake,98.0,70.0,3.99,22,20,England,8.8,Stout - Imperial Flavored / Pastry,...,0,0,0,0,0,0,0,0,0,0
1,Brouwerij De Molen (Swinkels Family Brewers),De Molen Bommen & Granaten Burgundy BA,NaN,NaN,4.13,7,7,Netherlands,11.1,Barley Wine / Wheat Wine / Rye Wine,...,0,0,0,0,0,0,0,1,0,0
2,Zagovor Brewery,Zagovor Sequoia Trail,NaN,NaN,3.50,1,1,Russia,6.0,IPA,...,0,0,0,0,0,0,1,0,0,0


## Step 3
Group beer_data specific categories into more generalized categories, to be used as dummy variables

In [8]:
# Too many features!

beer_data['beer_style'].nunique()

118

In [9]:
# Scrape https://www.ratebeer.com/beerstyles/ to find all general categories and specific categories
# Then, will create new column with general categories and dummy variables

import requests
from bs4 import BeautifulSoup as bs

url = 'https://www.ratebeer.com/beerstyles/'
response = requests.get(url)
status = response.status_code
if status == 200:
  page = response.text
  soup = bs(page)
else:
  print(f"Oops! Received status code {status}")

In [10]:
beer_dict = {}

for div in soup.find_all('div', class_='col-lg-4 col-xl-4 col-md-6'):
    beer_category = div.find('h3').text
    
    beer_name_list = []
    
    a_tags = [tag.find('a') for tag in div.find('ul').find_all('li')]
    
    for a in a_tags:
        beer_name = a.text
        
        beer_name_list.append(beer_name)
    
    beer_dict[beer_category] = beer_name_list
    
beer_dict

{'Anglo-American Ales': ['Altbier / Sticke Alt',
  'Barley Wine / Wheat Wine / Rye Wine',
  'Bitter - Ordinary / Best',
  'Bitter - Premium / Strong / Extra Special (ESB)',
  'Blonde Ale / Golden Ale',
  'Brown Ale',
  'Brown Ale - Imperial',
  'California Common / Steam Beer',
  'Cream Ale',
  'IIPA DIPA - Imperial / Double Hazy (NEIPA)',
  'IIPA DIPA - Imperial / Double IPA',
  'IPA',
  'IPA - Belgian / Saison',
  'IPA - Black / Brown / Cascadian Dark',
  'IPA - Brut',
  'IPA - English',
  'IPA - Flavored',
  'IPA - Hazy / New England (NEIPA)',
  'IPA - Milkshake / Smoothie',
  'IPA - Red',
  'IPA - Rye',
  'IPA - Sour / Wild',
  'IPA - White',
  'ISA - Session IPA',
  'Kölsch / Kölsch-Style',
  'Mild Ale',
  'Old Ale',
  'Pale Ale',
  'Pale Ale - American (APA)',
  'Pale Ale - Australian / New Zealand',
  'Pale Ale - Belgian / Farmhouse',
  'Pale Ale - English',
  'Pale Ale - Flavored',
  'Pale Ale - Hazy / New England',
  'Pale Ale - Rye',
  'Red Ale - Imperial',
  'Red Ale - Irish

In [11]:
# Make a flat list of tuples, including keys (broad category) repeated with each of its values (sub-category)
# Convert it back to a dictionary, but make the values (sub-category) the key!
# For every key (sub-category), change it to its corresponding value (category) in this new dictionary
# Code credit to Alex Baransky

beer_styles = []
for k, v in beer_dict.items():
    beer_styles.extend(list(zip([k]*len(v), v)))

beer_dict_swapped = {k:v for v, k in beer_styles}
beer_data['overall_style'] = beer_data['beer_style'].apply(lambda val: beer_dict_swapped[val] if val in beer_dict_swapped else None)

beer_data.head(3)

,brewery,beer_name,overall_score,style_score,rating,num_ratings,num_reviews,location,abv_percent,beer_style,...,served_in_stein,served_in_dimpled_mug,served_in_guinomi,served_in_weizen,served_in_yard,served_in_shaker,served_in_snifter,served_in_trappist_glass,served_in_stem_glass,overall_style
0,Siren Craft Brew,Siren Caribbean Chocolate Mole Cake,98.0,70.0,3.99,22,20,England,8.8,Stout - Imperial Flavored / Pastry,...,0,0,0,0,0,0,0,0,0,Stout and Porter
1,Brouwerij De Molen (Swinkels Family Brewers),De Molen Bommen & Granaten Burgundy BA,NaN,NaN,4.13,7,7,Netherlands,11.1,Barley Wine / Wheat Wine / Rye Wine,...,0,0,0,0,0,0,1,0,0,Anglo-American Ales
2,Zagovor Brewery,Zagovor Sequoia Trail,NaN,NaN,3.50,1,1,Russia,6.0,IPA,...,0,0,0,0,0,1,0,0,0,Anglo-American Ales


## Step 4
Calculate difference between date_added and day data was scraped

In [12]:
# Convert date_added to datetime
beer_data['date_added'] = pd.to_datetime(beer_data['date_added'])

# Create new column with all elements of series being day data was scraped
beer_data['date_scraped'] = pd.to_datetime('2022-2-6')

# Calculate difference between date_added and date_scraped, change to float
beer_data['days_on_site'] = (beer_data['date_scraped'] - beer_data['date_added']).dt.days

In [13]:
beer_data['days_on_site']

0         78.0
1         58.0
2          0.0
3       2410.0
4         40.0
         ...  
1480     239.0
1481       NaN
1482      39.0
1483     147.0
1484     554.0
Name: days_on_site, Length: 1485, dtype: float64

In [16]:
beer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485 entries, 0 to 1484
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   brewery                   1485 non-null   object        
 1   beer_name                 1485 non-null   object        
 2   overall_score             584 non-null    float64       
 3   style_score               584 non-null    float64       
 4   rating                    1485 non-null   float64       
 5   num_ratings               1485 non-null   int64         
 6   num_reviews               1485 non-null   int64         
 7   location                  1485 non-null   object        
 8   abv_percent               1476 non-null   float64       
 9   beer_style                1485 non-null   object        
 10  description               1208 non-null   object        
 11  served_in_list            1485 non-null   object        
 12  cal_per_355_ml      

## Step 5
Dropping unnecessary columns and NaN

In [17]:
# Drop these to avoid multicollinearity
beer_data.drop(columns=['overall_score', 'style_score'], inplace=True)

# Drop these because not appropriate for analyses, or used to create dummy variables only
beer_data.drop(columns=['brewery', 'beer_name', 'beer_style', 'description', 'served_in_list', 'date_added', 'url', 'date_scraped'], inplace=True)

In [18]:
beer_data.dropna(inplace=True)

## Step 6
Make remaining variables dummy variables (overall_style and location)

In [19]:
beer_data = pd.get_dummies(beer_data, drop_first=True)

In [20]:
beer_data.head()

,rating,num_ratings,num_reviews,abv_percent,cal_per_355_ml,served_in_lager_glass,served_in_thistle,served_in_flute,served_in_masu,served_in_english_pint,...,location_ Switzerland,location_ United States,location_ Wales,overall_style_Belgian-Style Ales,"overall_style_Cider, Mead, Sake",overall_style_Lagers,overall_style_Other Styles,overall_style_Sour Beer,overall_style_Stout and Porter,overall_style_Wheat Beer
0,3.99,22,20,8.8,264.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4.13,7,7,11.1,333.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.50,1,1,6.0,180.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.51,8,6,5.0,150.0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3.63,4,1,4.7,141.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [26]:
beer_data.to_pickle('beer_data_df')